# Przygotowanie

Przygotowanie Przed rozpoczęciem pracy z notatnikiem proszę zmienić jego nazwę dodając na początku numer albumu, imię i nazwisko. {nr_albumu}_{imię}_{nazwisko}_{nazwa}

Po wykonaniu wszystkich zadań proszę przesłać wypełniony notatnik przez platformę ELF za pomocą formularza "Prześlij projekt" w odpowiedniej sekcji.

# KNN

Podobnie jak w przypadku maszyny wektorów nosnych (SVC) oraz drzew decyzyjnych, KNN może słuzyc do rozwiazywania problemów zarówno klasyfikacji, jak i regresji. Algorytm KNN jest przykładem leniwego uczenia (lazy learning), co oznacza, że nie tworzy on modelu na podstawie danych uczących. Zamiast tego, w momencie klasyfikacji nowej obserwacji, algorytm poszukuje k najbardziej podobnych przypadków w zbiorze danych treningowych i przypisuje mu etykietę tej klasy, do której należy większość jego k sąsiadów.

**Algorytm KNN może być wykorzystywany zarówno do zadań klasyfikacji, jak i regresji.**

### Skalowanie danych

Jednym z istotnych elementów implementacji algorytmu KNN jest konieczność skalowania danych. Ponieważ algorytm ten opiera się na obliczaniu odległości pomiędzy punktami danych, różnice w jednostkach lub skali poszczególnych cech mogą prowadzić do zakłamanych wyników. Dlatego zazwyczaj skalowanie danych jest niezbędne, na przykład za pomocą standaryzacji lub normalizacji.

### Metryki odległości w KNN

Metryki odległości są kluczowym elementem algorytmu KNN, ponieważ definiują sposób działania algorytmu. Istnieją różne metryki odległości, z których najczęściej używanymi są:

1. *Odległość euklidesowa*: najpopularniejsza metryka odległości, która oblicza odległość między dwoma punktami w przestrzeni euklidesowej. W przypadku dwuwymiarowej przestrzeni (2D), odległość między punktami $(x_1, y_1)$ i $(x_2, y_2)$ obliczana jest jako $\sqrt{(x_2-x_1)^2 + (y_2-y_1)^2}$. W przestrzeni trójwymiarowej (3D), dodatkowo uwzględniana jest trzecia współrzędna.

2. *Odległość Manhattan*: oblicza sumę różnic wartości bezwzględnych między odpowiadającymi współrzędnymi punktów. W przestrzeni dwuwymiarowej, odległość między punktami $(x_1, y_1)$ i $(x_2, y_2)$ wynosi $|x_2-x_1| + |y_2-y_1|$, podczas gdy w przestrzeni trójwymiarowej dodaje się jeszcze różnicę trzeciej współrzędnej.


In [ ]:
import pandas as pd
import numpy as np

## Zadanie 1

Proszę pobrać, wczytać, oraz dokonać wizualizacji danych: https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic

Wizualizacji oraz dalszych obliczeń proszę dokonać tylko dla trzech wybranych przez siebie cech

In [ ]:
from ucimlrepo import fetch_ucirepo
import matplotlib.pyplot as plt
import seaborn as sns

breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)

X = breast_cancer_wisconsin_diagnostic.data.features
y = breast_cancer_wisconsin_diagnostic.data.targets
X = X.iloc[:, :3]

df = X.copy()
df['target'] = y
df['target'] = df['target'].map({'M': 'Malignant', 'B': 'Benign'})

sns.pairplot(df, hue='target', diag_kind='kde')
plt.suptitle("Pairplot of First 3 Features", y=1.02)
plt.show()

In [ ]:
import plotly.express as px

fig = px.scatter_3d(df,
                    x=df.columns[0],
                    y=df.columns[1],
                    z=df.columns[2],
                    color='target',
                    title="3D Scatter Plot of First 3 Features",
                    opacity=0.7)
fig.show()

## Zadanie 2

Proszę dokonać czyszczenia danych, sprawdzić czy nie ma danych brakujących oraz dokonać skalowania danych

In [ ]:
from sklearn.preprocessing import StandardScaler

print("missing values in data:")
print(X.isnull().sum())
print("missing values in labels:")
print(y.isnull().sum())

X = X.dropna()
y = y.dropna()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.iloc[:, :3])

df_scaled = pd.DataFrame(X_scaled, columns=X.columns[:3])

y_series = y['Diagnosis'].copy()

y_series = y_series.map({'M': 1, 'B': 0})
df_scaled['target'] = y_series
fig = px.scatter_3d(df_scaled,
                    x=df_scaled.columns[0],
                    y=df_scaled.columns[1],
                    z=df_scaled.columns[2],
                    color='target',
                    title="3D Scatter Plot (Standardized Features)",
                    opacity=0.7)
fig.show()

## Zadanie 3

Proszę zaimplementować algorytm KNN. Algorytm ma być w stanie dokonać klasyfikacji zarówno dla dwóch, jak i trzech cech. Parametr k ma być możliwy do podania na wejście algorytmu.

In [ ]:
from collections import Counter


def euclidean_distance(x1, x2):
    result = np.sqrt(np.sum((x1 - x2) ** 2))
    return result

def knn_pred(X_train, X_test, y_train , k, return_proba=False):
    predictions = []
    probabilities = []
    for x in X_test:
        distances = [euclidean_distance(x, x_train) for x_train in X_train]
        ind = np.argsort(distances)[:k]
        nearest_labels = [y_train.iloc[i] for i in ind]
        most_common = Counter(nearest_labels).most_common(1)
        predictions.append(most_common[0][0])
        count = Counter(nearest_labels)

        if return_proba:
            proba = count[1] / k
            probabilities.append(proba)

    if return_proba:
        return np.array(predictions), np.array(probabilities)
    else:
        return np.array(predictions)

def check_accuracy(X_train, X_test, y_train, y_test, k):
    y_pred = knn_pred(X_train, X_test, y_train, k)
    return np.mean(y_pred == y_test)


## Zadanie 4

Proszę wytrenować model z użyciem zbiotu danych. Proszę pamiętać o odpowienim podziale na zbiór testowy i treningowy. Klasyfikator powinien być trenowany na zbiorze treningowym, a wynik jego skuteczności po trenowaniu obliczany w oparciu o zbiór testowy.

Proszę przygotować wyniki, trenując algorytm z użyciem różnych parametrów k - należy przygotować wykresy (oś pionowa określa skuteczność, pozioma wartość parametru) pokazujące jak zmienia się skuteczność działania w zależności od zastosowanego parameteru k. Proszę o przygotowanie odpowiedniego porównania (tabela), co można zaobserwować?

In [ ]:
from sklearn.model_selection import train_test_split
from IPython.display import display
import time

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_series, test_size=0.3, random_state=123)

results = []
kk = range(1, 50)
start = time.time()
for k in kk:
    acc = check_accuracy(X_train, X_test, y_train, y_test, k)
    results.append({'k': k, 'accuracy': acc})
elapsed_time = time.time() - start
print(f"elapsed time = {elapsed_time} sec")

results_df = pd.DataFrame(results)
display(results_df)

plt.figure(figsize=(12, 8))
plt.plot(results_df['k'], results_df['accuracy'], marker='o')
plt.title("KNN accuracy depending on k")
plt.xlabel("num of neighbours - k")
plt.ylabel("accuracy")
plt.grid(True)
plt.xticks(kk)
plt.show()

'''Dla małych wartości k model może być bardziej podatny na szum (overfitting).
Dla dużych wartości k dokładność może się ustabilizować lub nieco spaść, jeśli sąsiedzi zawierają punkty z różnych klas.'''

## Zadanie 5

Proszę porównać działanie zaimplementowanego algorytmu z implementacją z Scikit-learn. Proszę dokonać porównania w oparciu o szybkość oraz skuteczność działania. Jakie wnioski można wyciągnać?

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

results = []
kk = range(1, 50)
start = time.time()
for k in kk:
    neigh = KNeighborsClassifier(n_neighbors = k,p=2)
    neigh.fit(X_train, y_train)
    y_pred = neigh.predict(X_test)
    acc = np.mean(y_pred == y_test)
    results.append({'k':k,'accuracy':acc})
elapsed_time = time.time() - start
print(f"elapsed time = {elapsed_time} sec")
results_df = pd.DataFrame(results)
display(results_df)

'''Skuteczność działania algorytmów jest taka sama jednak KNN z biblioteki jest znacznie szybszy.'''

## Zadanie 6

Proszę wyrysować krzywą ROC oraz obliczyć miarę AUC dla wytrenowanych modeli.

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier

knn_cls = KNeighborsClassifier(n_neighbors=13)
knn_cls.fit(X_train, y_train)

y_proba = knn_cls.predict_proba(X_test)[:, 1]

fpr, tpr, _ = roc_curve(y_test, y_proba, pos_label=1)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=1, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Scikit-learn KNN')
plt.legend(loc="lower right")
plt.grid()
plt.show()


In [ ]:
y_pred, y_proba = knn_pred(X_train, X_test, y_train , 13, return_proba=True)

fpr_my, tpr_my, _ = roc_curve(y_test, y_proba, pos_label=1)
roc_auc = auc(fpr_my, tpr_my)

plt.figure()
plt.plot(fpr_my, tpr_my, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=1, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - my implementation KNN')
plt.legend(loc="lower right")
plt.grid()
plt.show()